In [1]:
import cv2
import numpy as np
import math
import os
import pickle
from sklearn import svm
from skimage.feature import local_binary_pattern
from lib_vibe import vibe_gray
from pathlib import Path

In [2]:
def compute_lbp(arr):
    lbp = local_binary_pattern(arr, n_points, radius, METHOD)
    lbp = lbp.ravel()
    feature = np.zeros(n_bins)
    for i in lbp:
        feature[int(i)] += 1
    feature /= np.linalg.norm(feature, ord=1)
    return feature

In [3]:
def toHSI(frame):
    hsi_img = frame.copy()
    por = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV) 

    B = frame[:, :, 0 ]
    G = frame[:, :, 1 ] 
    R = frame[:, :, 2 ] 

    H = por[:, :, 0 ]
    S = por[:, :, 1 ] 
    V = por[:, :, 2 ]

    [B,G,R] = [i/255.0  for i in ([B,G,R])]
    I = (R + G + B)/3.0

    hsi_img[:, :, 0 ] = H
    hsi_img[:,:, 1 ] = S
    hsi_img[:,:, 2 ] = I* 255 
    return hsi_img

In [52]:
def videoProc(path, folderType):
    c=1
    frame_index = 0
    framelistgray = []
    framelist = []
    framemem = []

    lower = [0, 51, 100]
    upper = [30, 255, 255]
    lower = np.array(lower, dtype='uint8')
    upper = np.array(upper, dtype='uint8')

    cap = cv2.VideoCapture(path)
    folder = Path(path).stem

    while (cap.isOpened()):
        ret, frame = cap.read()
        if ret == True:
            frame = cv2.resize(frame, (vwidth, vheight))
            framegray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            if len(framemem) < 120 :
                framemem.append(frame)
            elif len(framemem) == 120 :
                framemem.append(frame)
                framemem.pop(0)

            if c%15==0:
                if len(framelistgray)<3 :
                    framelistgray.append(framegray)
                    framelist.append(frame)
                elif len(framelistgray) == 3 :
                    framelistgray.append(framegray)
                    framelistgray.pop(0)
                    framelist.append(frame)
                    framelist.pop(0)

                    frame_diff = cv2.absdiff(framelistgray[2], framelistgray[1])
                    frame_diff = cv2.threshold(frame_diff, 25, 255, cv2.THRESH_BINARY)[1]
                    frame_diff2 = cv2.absdiff(framelistgray[1], framelistgray[0])
                    frame_diff2 = cv2.threshold(frame_diff2, 25, 255, cv2.THRESH_BINARY)[1]
                    framediff3 = cv2.bitwise_and(frame_diff, frame_diff2)

                    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (10, 10))
                    kernel2 = cv2.getStructuringElement(cv2.MORPH_RECT, (20, 20))
                    kernel3 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))
                    closing = cv2.morphologyEx(framediff3, cv2.MORPH_CLOSE, kernel)

                    cv2.imshow('3 frame diff morph', closing)
                    
                    image_copy6 = framelist[1].copy() 
                    filename3df = f'./framecap/training/{folderType}/{folder}/6. 3Framediff/frame{c-5} BGR.jpg'
                    filenamemask3df = f'./framecap/training/{folderType}/{folder}/6. 3Framediff/frame{c-5} Mask.jpg'
                    os.makedirs(os.path.dirname(filename3df), exist_ok=True)
                    os.makedirs(os.path.dirname(filenamemask3df), exist_ok=True)
                    cv2.imwrite(filename3df, image_copy6)
                    cv2.imwrite(filenamemask3df, closing)

                    if frame_index == 0:
                        vibe.AllocInit(frame_diff2)
                    segmentation_map = vibe.Segmentation(frame_diff2)
                    vibe.Update(frame_diff2, segmentation_map)

                    cv2.imshow('Vibe Segmentation', segmentation_map)
                    image_copy7 = framelist[1].copy()
                    filenameViberaw = f'./framecap/training/{folderType}/{folder}/7. Vibe RAW/frame{c-5} BGR.jpg'
                    filenamemaskViberawmask = f'./framecap/training/{folderType}/{folder}/7. Vibe RAW/frame{c-5} Mask.jpg'
                    os.makedirs(os.path.dirname(filenameViberaw), exist_ok=True)
                    os.makedirs(os.path.dirname(filenamemaskViberawmask), exist_ok=True)
                    cv2.imwrite(filenameViberaw, image_copy7)
                    cv2.imwrite(filenamemaskViberawmask, segmentation_map)
                    
                    
                    framevibe = cv2.bitwise_and(segmentation_map, closing)
                    framevibeDilate = cv2.dilate(framevibe,kernel,iterations = 1)
                    cv2.imshow('vibeframe', framevibeDilate)
                    contours, hierarchy = cv2.findContours(image=framevibeDilate, mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_NONE)
                    
                    image_copy5 = framelist[1].copy()
                    filenameVibe = f'./framecap/training/{folderType}/{folder}/5. Vibe/frame{c-5} BGR.jpg'
                    filenamemaskVibe = f'./framecap/training/{folderType}/{folder}/5. Vibe/frame{c-5} Mask.jpg'
                    os.makedirs(os.path.dirname(filenameVibe), exist_ok=True)
                    os.makedirs(os.path.dirname(filenamemaskVibe), exist_ok=True)
                    cv2.imwrite(filenameVibe, image_copy5)
                    cv2.imwrite(filenamemaskVibe, framevibeDilate)
                    
                    image_copy = framelist[1].copy()
                    image_copy2 = framelist[1].copy()
                    resultbgr = framelist[1].copy()
                    resultbgr = cv2.bitwise_and(resultbgr, resultbgr, mask=framevibeDilate)
                    result = toHSI(resultbgr)
                    maskhsi = cv2.inRange(result, lower, upper)
                    closinghsi = cv2.morphologyEx(maskhsi, cv2.MORPH_CLOSE, kernel)
                    dilatehsi = cv2.dilate(closinghsi,kernel3,iterations = 2)
                    output = cv2.bitwise_and(resultbgr, result, mask=dilatehsi)
                    outputbgr = cv2.bitwise_and(resultbgr, resultbgr, mask=dilatehsi)
                    cv2.imshow('maskHSI', maskhsi)
                    cv2.imshow('HSI', output)
                    cv2.imshow('dilated HSI mask', dilatehsi)
                    firecontours, firehierarchy = cv2.findContours(image=dilatehsi, mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_NONE)
                    
                    filenamemaskHSI = f'./framecap/training/{folderType}/{folder}/4. HSI contours/frame{c-5} maskHSI.jpg'
                    os.makedirs(os.path.dirname(filenamemaskHSI), exist_ok=True)
                    cv2.imwrite(filenamemaskHSI, output)
                    
                    iterframe = 0
                    for iImg, cnt in enumerate(firecontours):
                        image_copy3 = framelist[1].copy()
                        image_copy4 = framelist[1].copy()
                        if cv2.contourArea(cnt) > 125 :
                            newframemem = framemem.copy()
                            x,y,w,h = cv2.boundingRect(cnt)
                            cropped = output[y:y+h, x:x+w]
                            croppedbgr = outputbgr[y:y+h, x:x+w]
                            cv2.imshow("BGR Cropped", croppedbgr)
                            cv2.imshow("HSI Cropped", cropped)
                            
                            filenameHSI = f'./framecap/training/{folderType}/{folder}/4. HSI contours/frame{c-5} HSI{iterframe}.jpg'
                            
                            os.makedirs(os.path.dirname(filenameHSI), exist_ok=True)
                            

                            cv2.rectangle(image_copy4,(x,y),(x+w,y+h),(0,255,0),2)
                            cv2.putText(image_copy4,'HSI Contour position',(x+w+10,y+h),0,0.3,(0,255,0))
#                                 cv2.drawContours(image=image_copy3, contours=cnt, contourIdx=-1, color=(0, 255, 0), thickness=2, lineType=cv2.LINE_AA)
                            cv2.imwrite(filenameHSI, image_copy4)
                            
                            
                            for idx, framevalue in enumerate(newframemem):
                                newframemem[idx] = newframemem[idx][y:y+h, x:x+w]
                                cv2.imshow('newframemem', newframemem[idx])

                            hf, wf, _ = newframemem[0].shape
                            centerw = math.ceil(wf/2)
                            centerh = math.ceil(hf/2)

                            XY = newframemem[0]
         
                            XT = newframemem[0][centerh:centerh+1, 0:wf]
                            for idx2 in range(0, len(newframemem)):
                                XT = cv2.vconcat([XT, newframemem[idx2][centerh:centerh+1, 0:wf]])

                            YT = newframemem[0][0:hf, centerw:centerw+1]
                            for idx2 in range(0, len(newframemem)):
                                YT = cv2.hconcat([YT, newframemem[idx2][0:hf, centerw:centerw+1]])
                                
                            XYgray = cv2.cvtColor(XY, cv2.COLOR_BGR2GRAY)
                            XTgray = cv2.cvtColor(XT, cv2.COLOR_BGR2GRAY)
                            YTgray = cv2.cvtColor(YT, cv2.COLOR_BGR2GRAY)

                            XYhist = compute_lbp(XYgray)
                            XThist = compute_lbp(XTgray)
                            YThist = compute_lbp(YTgray)

                            XYhist = XYhist.tolist()
                            XThist = XThist.tolist()
                            YThist = YThist.tolist()

                            filenameXT = f'./framecap/training/{folderType}/{folder}/3. Plane/XT/frame{c-5} XT{iterframe}.jpg'
                            filenameXY = f'./framecap/training/{folderType}/{folder}/3. Plane/XY/frame{c-5} XY{iterframe}.jpg'
                            filenameYT = f'./framecap/training/{folderType}/{folder}/3. Plane/YT/frame{c-5} YT{iterframe}.jpg'
                            os.makedirs(os.path.dirname(filenameXT), exist_ok=True)
                            os.makedirs(os.path.dirname(filenameXY), exist_ok=True)   
                            os.makedirs(os.path.dirname(filenameYT), exist_ok=True)   
                
                            cv2.imwrite(filenameXY, XY)
                            cv2.imwrite(filenameXT, XT)
                            cv2.imwrite(filenameYT, YT)

                            cntplane = XYhist + XThist + YThist
                            
                            txtplane = open(f"./framecap/training/{folderType}/{folder}/3. Plane/!PlaneInfo.txt", "a") 
                            txtplane.write(f"Frame: {c-5}\nContour: {iterframe}\nXY histogram: {XYhist}\nXT histogram: {XThist}\nYT histogram: {YThist}\n\n") 
                            txtplane.close()
                            
                            iterframe += 1
                            plane.append(cntplane)

                    cv2.drawContours(image=image_copy, contours=contours, contourIdx=-1, color=(0, 255, 0), thickness=2, lineType=cv2.LINE_AA)
                    cv2.drawContours(image=image_copy2, contours=firecontours, contourIdx=-1, color=(0, 255, 0), thickness=2, lineType=cv2.LINE_AA)
                    cv2.imshow('Background Substraction Contour', image_copy)
                    cv2.imshow('Fire Contour', image_copy2)
                    frame_index += 1

            cv2.imshow('Actual Frame!', frame)  
            c+=1
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            break
    cap.release()
    cv2.destroyAllWindows()
    return plane

In [58]:
def firepredict(path, folderType):
    c=1
    frame_index = 0
    framelistgray = []
    framelist = []
    framemem = []

    lower = [0, 51, 100]
    upper = [30, 255, 255]
    lower = np.array(lower, dtype='uint8')
    upper = np.array(upper, dtype='uint8')
    
    cap = cv2.VideoCapture(path)
    folder = Path(path).stem
    
    while (cap.isOpened()):
        ret, frame = cap.read()
        if ret == True:
            frame = cv2.resize(frame, (vwidth, vheight))
            framegray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            if len(framemem) < 120 :
                framemem.append(frame)
            elif len(framemem) == 120 :
                framemem.append(frame)
                framemem.pop(0)

            if c%15==0:
                if len(framelistgray)<3 :
                    framelistgray.append(framegray)
                    framelist.append(frame)
                elif len(framelistgray) == 3 :
                    framelistgray.append(framegray)
                    framelistgray.pop(0)
                    framelist.append(frame)
                    framelist.pop(0)

                    frame_diff = cv2.absdiff(framelistgray[2], framelistgray[1])
                    frame_diff = cv2.threshold(frame_diff, 25, 255, cv2.THRESH_BINARY)[1]
                    frame_diff2 = cv2.absdiff(framelistgray[1], framelistgray[0])
                    frame_diff2 = cv2.threshold(frame_diff2, 25, 255, cv2.THRESH_BINARY)[1]
                    framediff3 = cv2.bitwise_and(frame_diff, frame_diff2)

                    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (10, 10))
                    kernel2 = cv2.getStructuringElement(cv2.MORPH_RECT, (20, 20))
                    kernel3 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))
                    closing = cv2.morphologyEx(framediff3, cv2.MORPH_CLOSE, kernel)

                    cv2.imshow('3 frame diff morph', closing)
                    
                    image_copy6 = framelist[1].copy() 
                    filename3df = f'./framecap/testing/{folderType}/{folder}/6. 3Framediff/frame{c-5} BGR.jpg'
                    filenamemask3df = f'./framecap/testing/{folderType}/{folder}/6. 3Framediff/frame{c-5} Mask.jpg'
                    os.makedirs(os.path.dirname(filename3df), exist_ok=True)
                    os.makedirs(os.path.dirname(filenamemask3df), exist_ok=True)
                    cv2.imwrite(filename3df, image_copy6)
                    cv2.imwrite(filenamemask3df, closing)

                    if frame_index == 0:
                        vibe.AllocInit(frame_diff2)
                    segmentation_map = vibe.Segmentation(frame_diff2)
                    vibe.Update(frame_diff2, segmentation_map)

                    cv2.imshow('Vibe Segmentation', segmentation_map)
                    image_copy7 = framelist[1].copy()
                    filenameViberaw = f'./framecap/testing/{folderType}/{folder}/7. Vibe RAW/frame{c-5} BGR.jpg'
                    filenamemaskViberawmask = f'./framecap/testing/{folderType}/{folder}/7. Vibe RAW/frame{c-5} Mask.jpg'
                    os.makedirs(os.path.dirname(filenameViberaw), exist_ok=True)
                    os.makedirs(os.path.dirname(filenamemaskViberawmask), exist_ok=True)
                    cv2.imwrite(filenameViberaw, image_copy7)
                    cv2.imwrite(filenamemaskViberawmask, segmentation_map)
                    
                    framevibe = cv2.bitwise_and(segmentation_map, closing)
                    framevibeDilate = cv2.dilate(framevibe,kernel,iterations = 1)
                    cv2.imshow('vibeframe', framevibeDilate)
                    contours, hierarchy = cv2.findContours(image=framevibeDilate, mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_NONE)
                    
                    image_copy5 = framelist[1].copy()
                    filenameVibe = f'./framecap/testing/{folderType}/{folder}/5. Vibe/frame{c-5} BGR.jpg'
                    filenamemaskVibe = f'./framecap/testing/{folderType}/{folder}/5. Vibe/frame{c-5} Mask.jpg'
                    os.makedirs(os.path.dirname(filenameVibe), exist_ok=True)
                    os.makedirs(os.path.dirname(filenamemaskVibe), exist_ok=True)
                    cv2.imwrite(filenameVibe, image_copy5)
                    cv2.imwrite(filenamemaskVibe, framevibeDilate)
                        
                    image_copy = framelist[1].copy()
                    image_copy2 = framelist[1].copy()
                    resultbgr = framelist[1].copy()
                    
                    resultbgr = cv2.bitwise_and(resultbgr, resultbgr, mask=framevibeDilate)
                    result = toHSI(resultbgr)
                    maskhsi = cv2.inRange(result, lower, upper)
                    closinghsi = cv2.morphologyEx(maskhsi, cv2.MORPH_CLOSE, kernel)
                    dilatehsi = cv2.dilate(closinghsi,kernel3,iterations = 2)
                    output = cv2.bitwise_and(resultbgr, result, mask=dilatehsi)
                    outputbgr = cv2.bitwise_and(resultbgr, resultbgr, mask=dilatehsi)
                    cv2.imshow('HSI', output)
                    cv2.imshow('maskHSI', dilatehsi)
                    firecontours, firehierarchy = cv2.findContours(image=dilatehsi, mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_NONE)
                    
                    filenamemaskHSI = f'./framecap/testing/{folderType}/{folder}/4. HSI contours/frame{c-5} maskHSI.jpg'
                    os.makedirs(os.path.dirname(filenamemaskHSI), exist_ok=True)
                    cv2.imwrite(filenamemaskHSI, output)
                    
                    iterframe = 0
                    for iImg, cnt in enumerate(firecontours):
                        image_copy3 = framelist[1].copy()
                        image_copy4 = framelist[1].copy()
                        if cv2.contourArea(cnt) > 125 :
                            newframemem = framemem.copy()
                            x,y,w,h = cv2.boundingRect(cnt)
                            cropped = output[y:y+h, x:x+w]
                            croppedbgr = outputbgr[y:y+h, x:x+w]
                            cv2.imshow("BGR Cropped", croppedbgr)
                            cv2.imshow("HSI Cropped", cropped)
                            
                            filenameHSI = f'./framecap/testing/{folderType}/{folder}/4. HSI contours/frame{c-5} HSI{iterframe}.jpg'
                            os.makedirs(os.path.dirname(filenameHSI), exist_ok=True)
                            cv2.rectangle(image_copy4,(x,y),(x+w,y+h),(0,255,0),2)
                            cv2.putText(image_copy4,'HSI Contour position',(x+w+10,y+h),0,0.3,(0,255,0))
#                                 cv2.drawContours(image=image_copy3, contours=cnt, contourIdx=-1, color=(0, 255, 0), thickness=2, lineType=cv2.LINE_AA)
                            cv2.imwrite(filenameHSI, image_copy4)
                            
                            for idx, framevalue in enumerate(newframemem):
                                newframemem[idx] = newframemem[idx][y:y+h, x:x+w]
                                cv2.imshow('newframemem', newframemem[idx])

                            hf, wf, _ = newframemem[0].shape
                            centerw = math.ceil(wf/2)
                            centerh = math.ceil(hf/2)

                            XY = newframemem[0]
         
                            XT = newframemem[0][centerh:centerh+1, 0:wf]
                            for idx2 in range(0, len(newframemem)):
                                XT = cv2.vconcat([XT, newframemem[idx2][centerh:centerh+1, 0:wf]])

                            YT = newframemem[0][0:hf, centerw:centerw+1]
                            for idx2 in range(0, len(newframemem)):
                                YT = cv2.hconcat([YT, newframemem[idx2][0:hf, centerw:centerw+1]])
                                
                            XYgray = cv2.cvtColor(XY, cv2.COLOR_BGR2GRAY)
                            XTgray = cv2.cvtColor(XT, cv2.COLOR_BGR2GRAY)
                            YTgray = cv2.cvtColor(YT, cv2.COLOR_BGR2GRAY)

                            XYhist = compute_lbp(XYgray)
                            XThist = compute_lbp(XTgray)
                            YThist = compute_lbp(YTgray)

                            XYhist = XYhist.tolist()
                            XThist = XThist.tolist()
                            YThist = YThist.tolist()
    
                            filenameXT = f'./framecap/testing/{folderType}/{folder}/3. Plane/XT/frame{c-5} XT{iterframe}.jpg'
                            filenameXY = f'./framecap/testing/{folderType}/{folder}/3. Plane/XY/frame{c-5} XY{iterframe}.jpg'
                            filenameYT = f'./framecap/testing/{folderType}/{folder}/3. Plane/YT/frame{c-5} YT{iterframe}.jpg'
                            os.makedirs(os.path.dirname(filenameXT), exist_ok=True)
                            os.makedirs(os.path.dirname(filenameXY), exist_ok=True)   
                            os.makedirs(os.path.dirname(filenameYT), exist_ok=True)   
                
                            cv2.imwrite(filenameXY, XY)
                            cv2.imwrite(filenameXT, XT)
                            cv2.imwrite(filenameYT, YT)

                            cntplane = XYhist + XThist + YThist
                            
                            txtplane = open(f"./framecap/testing/{folderType}/{folder}/3. Plane/!PlaneInfo.txt", "a") 
                            txtplane.write(f"Frame: {c-5}\nContour: {iterframe}\nXY histogram: {XYhist}\nXT histogram: {XThist}\nYT histogram: {YThist}\n\n") 
                            txtplane.close()
                            
                            npcnt = np.array(cntplane)
                            predresult = clf.predict([npcnt])
                            print(predresult)
                            if predresult == 0 :
                                filename = f'./framecap/testing/{folderType}/{folder}/1. Fire Detected/frame{c-5} firedetected{iterframe}.jpg'
                                os.makedirs(os.path.dirname(filename), exist_ok=True)
                                
                                cv2.rectangle(image_copy3,(x,y),(x+w,y+h),(0,255,0),2)
                                cv2.putText(image_copy3,'Fire Detected',(x+w+10,y+h),0,0.3,(0,255,0))
#                                 cv2.drawContours(image=image_copy3, contours=cnt, contourIdx=-1, color=(0, 255, 0), thickness=2, lineType=cv2.LINE_AA)
                                cv2.imwrite(filename, image_copy3)
                                cv2.imshow("Label Fire", image_copy3)
                            elif predresult == 1 :
                                filename = f'./framecap/testing/{folderType}/{folder}/2. NonFire Detected/frame{c-5} NOTfiredetected{iterframe}.jpg'
                                os.makedirs(os.path.dirname(filename), exist_ok=True)
                        
                                cv2.rectangle(image_copy3,(x,y),(x+w,y+h),(0,255,0),2)
                                cv2.putText(image_copy3,'Not Fire',(x+w+10,y+h),0,0.3,(0,255,0))
                                cv2.imwrite(filename, image_copy3)
                                cv2.imshow("Label Non Fire", image_copy3)
                            
                            iterframe += 1
                            plane.append(cntplane)
                        
                    cv2.drawContours(image=image_copy, contours=contours, contourIdx=-1, color=(0, 255, 0), thickness=2, lineType=cv2.LINE_AA)
                    cv2.drawContours(image=image_copy2, contours=firecontours, contourIdx=-1, color=(0, 255, 0), thickness=2, lineType=cv2.LINE_AA)
                    cv2.imshow('Background Substraction Contour', image_copy)
                    cv2.imshow('Fire Contour', image_copy2)
                    frame_index += 1

            cv2.imshow('Actual Frame!', frame)  
            c+=1
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            break
    cap.release()
    cv2.destroyAllWindows()
    return plane

In [20]:
radius = 2
n_points = 8*radius
METHOD = "uniform"
n_bins = n_points + 2

vwidth = 480
vheight = 270

vibe = vibe_gray()
fireLBPfeature = []
nonfireLBPfeature = []
testingfire = []
testingnonfire = []

### Code Test

In [63]:
plane = []
folderType = "Fire"
path=os.path.join("./dataset/testing/Fire/Telyu - 4.m4v")
x = firepredict(path, folderType)
fireLBPfeature = x
# print(fireLBPfeature[0])
# print(len(fireLBPfeature))
# print(len(fireLBPfeature[0]))

AllocInit!
270 480
[0.]
[1.]
[0.]
[1.]
[1.]
[0.]
[1.]
[1.]
[0.]
[1.]
[0.]
[1.]
[1.]
[0.]
[1.]
[1.]
[0.]
[1.]
[0.]
[1.]
[1.]
[1.]
[0.]
[1.]
[0.]
[1.]
[0.]
[1.]
[1.]
[0.]
[1.]
[1.]
[0.]
[1.]
[0.]
[1.]
[1.]
[0.]
[1.]
[1.]
[0.]
[1.]
[0.]
[1.]
[0.]
[1.]
[0.]
[1.]
[0.]
[1.]
[0.]
[1.]
[0.]
[1.]
[0.]
[1.]
[0.]
[1.]
[0.]
[0.]
[0.]
[1.]
[1.]
[0.]
[1.]
[1.]
[0.]
[1.]
[0.]
[1.]
[0.]
[1.]
[0.]
[1.]
[0.]
[1.]
[0.]
[1.]
[1.]
[0.]
[0.]
[1.]
[0.]
[1.]
[1.]
[0.]
[0.]
[1.]
[1.]
[1.]
[1.]
[1.]
[0.]
[1.]
[1.]
[1.]
[1.]
[0.]
[1.]
[0.]
[0.]
[0.]
[1.]
[1.]
[0.]
[1.]
[1.]
[1.]
[1.]
[1.]
[0.]
[0.]
[0.]
[0.]
[1.]
[0.]
[0.]
[1.]
[1.]
[1.]
[0.]
[1.]
[1.]
[0.]
[0.]
[1.]
[0.]
[0.]
[1.]
[1.]


In [61]:
plane = []
folderType = "Non Fire"
path=os.path.join("./dataset/testing/Non Fire/Telyu - 887.m4v")
x = firepredict(path, folderType)
fireLBPfeature = x
# print(fireLBPfeature[0])
# print(len(fireLBPfeature))
# print(len(fireLBPfeature[0]))

AllocInit!
270 480
[1.]
[1.]


## Training

In [21]:
folderType = "Fire"
for file in os.listdir("./dataset/training/Fire/"):
    if file.endswith(".mp4") or file.endswith(".m4v") or file.endswith(".avi"):
        plane = []
        path=os.path.join("./dataset/training/Fire/", file)
        x = videoProc(path, folderType)
        fireLBPfeature = fireLBPfeature + x

AllocInit!
270 480
AllocInit!
270 480
AllocInit!
270 480
AllocInit!
270 480
AllocInit!
270 480
AllocInit!
270 480
AllocInit!
270 480
AllocInit!
270 480
AllocInit!
270 480
AllocInit!
270 480
AllocInit!
270 480
AllocInit!
270 480
AllocInit!
270 480
AllocInit!
270 480
AllocInit!
270 480
AllocInit!
270 480
AllocInit!
270 480
AllocInit!
270 480
AllocInit!
270 480
AllocInit!
270 480
AllocInit!
270 480
AllocInit!
270 480
AllocInit!
270 480
AllocInit!
270 480
AllocInit!
270 480
AllocInit!
270 480
AllocInit!
270 480


In [22]:
folderType = "Non Fire"
for file in os.listdir("./dataset/training/Non Fire/"):
    if file.endswith(".mp4") or file.endswith(".m4v") or file.endswith(".avi"):
        plane = []
        path=os.path.join("./dataset/training/Non Fire/", file)
        y = videoProc(path, folderType)
        nonfireLBPfeature = nonfireLBPfeature + y

AllocInit!
270 480
AllocInit!
270 480
AllocInit!
270 480
AllocInit!
270 480
AllocInit!
270 480
AllocInit!
270 480
AllocInit!
270 480
AllocInit!
270 480
AllocInit!
270 480
AllocInit!
270 480
AllocInit!
270 480
AllocInit!
270 480
AllocInit!
270 480
AllocInit!
270 480
AllocInit!
270 480
AllocInit!
270 480
AllocInit!
270 480
AllocInit!
270 480
AllocInit!
270 480
AllocInit!
270 480
AllocInit!
270 480
AllocInit!
270 480
AllocInit!
270 480
AllocInit!
270 480
AllocInit!
270 480
AllocInit!
270 480
AllocInit!
270 480
AllocInit!
270 480
AllocInit!
270 480
AllocInit!
270 480
AllocInit!
270 480
AllocInit!
270 480
AllocInit!
270 480
AllocInit!
270 480
AllocInit!
270 480
AllocInit!
270 480
AllocInit!
270 480
AllocInit!
270 480
AllocInit!
270 480
AllocInit!
270 480


In [23]:
# print(fireLBPfeature)
print(len(fireLBPfeature))
print(len(fireLBPfeature[0]))

# print(nonfireLBPfeature)
print(len(nonfireLBPfeature))
print(len(nonfireLBPfeature[0]))

5325
54
15154
54


In [24]:
Xtrain = fireLBPfeature + nonfireLBPfeature
Xtrain = np.array(Xtrain)

In [25]:
len(Xtrain)

20479

In [26]:
Ytrain = np.empty(shape=[0])
for i in range(len(fireLBPfeature)):
    Ytrain = np.append(Ytrain, 0)
for i in range(len(nonfireLBPfeature)):
    Ytrain = np.append(Ytrain, 1)

In [27]:
len(Ytrain)

20479

## Classification
0 = Fire <br>
1 = Non Fire

In [28]:
clf = svm.SVC(kernel = 'linear', C=1)

In [29]:
clf.fit(Xtrain, Ytrain)

SVC(C=1, kernel='linear')

### Single Testing

In [17]:
plane = []
folderType = "Fire"
path=os.path.join("./dataset/testing/Fire/Yusron - 9922.mp4")
z = firepredict(path, folderType)
res = clf.predict(z)
print(res)
print("Total frame :", len(res))
print("Fire total:", np.count_nonzero(res == 0))
print("Non Fire total:", np.count_nonzero(res == 1))

AllocInit!
270 480
[1.]
[1.]
[1.]
[0.]
[1.]
[0.]
[1.]
[1.]
[0.]
[1.]
[1.]
[0.]
[1.]
[1.]
[1.]
[0.]
[1.]
[1.]
[0.]
[1.]
[1.]
[0.]
[0.]
[0.]
[1.]
[1.]
[1.]
[0.]
[1.]
[1.]
[0.]
[0.]
[1.]
[1.]
[1.]
[0.]
[0.]
[1.]
[1.]
[0.]
[0.]
[1.]
[1.]
[1.]
[0.]
[1.]
[1.]
[0.]
[0.]
[1.]
[1.]
[1.]
[0.]
[0.]
[1.]
[1.]
[1.]
[0.]
[0.]
[1.]
[1.]
[0.]
[0.]
[1.]
[1.]
[1.]
[1.]
[0.]
[1.]
[1.]
[1.]
[0.]
[0.]
[1.]
[0.]
[1.]
[1.]
[0.]
[0.]
[1.]
[1.]
[1.]
[0.]
[0.]
[0.]
[1.]
[1.]
[1.]
[0.]
[0.]
[0.]
[1.]
[1.]
[1.]
[0.]
[0.]
[1.]
[1.]
[1.]
[0.]
[1.]
[1.]
[1.]
[0.]
[1.]
[1.]
[1.]
[0.]
[0.]
[1.]
[1.]
[1.]
[0.]
[1.]
[1.]
[0.]
[0.]
[0.]
[1.]
[1.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[1.]
[0.]
[1.]
[0.]
[1.]
[1.]
[0.]
[1.]
[1.]
[1.]
[0.]
[1.]
[0.]
[1.]
[1.]
[0.]
[1.]
[1.]
[0.]
[1.]
[1.]
[0.]
[1.]
[1.]
[0.]
[1.]
[0.]
[1.]
[0.]
[

In [25]:
plane = []
path=os.path.join("./dataset/testing/Non Fire/Yusron - 8813.mp4")
z = firepredict(path)
res = clf.predict(z)
print(res)
print("Total frame :", len(res))
print("Fire total:", np.count_nonzero(res == 0))
print("Non Fire total:", np.count_nonzero(res == 1))

AllocInit!
270 480
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Total frame : 18
Fire total: 0
Non Fire total: 18


### Group Testing

In [32]:
folderType = "Fire"
for file in os.listdir("./dataset/testing/Fire/"):
    if file.endswith(".mp4") or file.endswith(".m4v") or file.endswith(".avi"):
        plane = []
        path=os.path.join("./dataset/testing/Fire/", file)
        fp = firepredict(path, folderType)
        if len(fp) != 0 :
            fpres = clf.predict(fp)
            print(fpres)
            print("Total frame :", len(fpres))
            print("Fire total:", np.count_nonzero(fpres == 0))
            print("Non Fire total:", np.count_nonzero(fpres == 1))
        else :
            print("Empty")
        
        folder = Path(path).stem
        filename = f"./framecap/testing/{folderType}/{folder}/"
        os.makedirs(os.path.dirname(filename), exist_ok=True)
        txtinfo = open(f"./framecap/testing/{folderType}/{folder}/!Info.txt", "w") 
        txtinfo.write(f"{fpres}\n\nTotal frame: {len(fpres)}\nFire total: {np.count_nonzero(fpres == 0)}\nNon Fire total: {np.count_nonzero(fpres == 1)}") 
        txtinfo.close() 

AllocInit!
270 480
[1.]
[0.]
[0.]
[1.]
[0.]
[1.]
[0.]
[1.]
[0.]
[1.]
[1.]
[1.]
[0.]
[1.]
[0.]
[1.]
[0.]
[0.]
[0.]
[0.]
[0.]
[1.]
[0.]
[0.]
[1.]
[1.]
[1.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[0.]
[0.]
[1.]
[0.]
[1.]
[1.]
[0.]
[0.]
[0.]
[0.]
[1.]
[0.]
[0.]
[0.]
[0.]
[1.]
[1.]
[0.]
[1.]
[0.]
[1.]
[0.]
[0.]
[0.]
[0.]
[1.]
[1.]
[1.]
[0.]
[1.]
[0.]
[1.]
[1.]
[1.]
[0.]
[1.]
[1.]
[1.]
[1.]
[0.]
[1.]
[0.]
[0.]
[0.]
[1.]
[0.]
[0.]
[1.]
[0.]
[1.]
[1.]
[1.]
[0.]
[1.]
[0.]
[1.]
[0.]
[1.]
[0.]
[1.]
[0.]
[0.]
[0.]
[1.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[1.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[1.]
[0.]
[1.]
[0.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[0.]
[0.]
[1.]
[0.]
[1.]
[0.]
[1.]
[0.]
[1.]
[1.]
[1.]
[1.]
[0.]
[1.]
[1.]
[1.]
[0.]
[0.]
[1.]
[0.]
[0.]
[0.]
[0.]
[1.]
[0.]
[1.]
[0.]
[0.]
[0.]
[0.]
[1.]
[1.]
[1.]
[0.]
[0.]
[1.]
[0.]
[0.]
[1.]
[0.]
[1.]
[0.]
[0.]
[0.]
[0.]
[

KeyboardInterrupt: 

In [31]:
folderType = "Non Fire"
for file in os.listdir("./dataset/testing/Non Fire/"):
    if file.endswith(".mp4") or file.endswith(".m4v") or file.endswith(".avi"):
        plane = []
        path=os.path.join("./dataset/testing/Non Fire/", file)
        nfp = firepredict(path, folderType)
        if len(nfp) != 0 :
            nfpres = clf.predict(nfp)
            print(nfpres)
            print("Total frame :", len(nfpres))
            print("Fire total:", np.count_nonzero(nfpres == 0))
            print("Non Fire total:", np.count_nonzero(nfpres == 1))
        else :
            print("Empty")
        
        folder = Path(path).stem
        filename = f"./framecap/testing/{folderType}/{folder}/"
        os.makedirs(os.path.dirname(filename), exist_ok=True)
        txtinfo = open(f"./framecap/testing/{folderType}/{folder}/!Info.txt", "w") 
        txtinfo.write(f"{nfpres}\n\nTotal frame: {len(nfpres)}\nFire total: {np.count_nonzero(nfpres == 0)}\nNon Fire total: {np.count_nonzero(nfpres == 1)}") 
        txtinfo.close() 

AllocInit!
270 480
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[0.]
[1.]
[1.]
[1.]
[0.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[0.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[0.]
[1.]
[1.]
[1.]
[0.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[0.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[

AllocInit!
270 480
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[0.]
[1.]
[0.]
[0.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[0.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[0.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[0.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[

[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1

[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[0.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[0.]
[1.]
[1.]
[1. 1. 1. ... 0. 1. 1.]
T